# Basic RAG Demonstration


## 1. Loading Data


In [1]:
import sys
import time
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

In [2]:
from src.load_data import load_markdown_files
from src.chunking import chunk_documents
from src.embedder import generate_embeddings, generate_query_embedding
from src.retriever import retrieve
from src.llm_orchestrator import generate_answer

DATA_DIR = PROJECT_ROOT / 'data' / 'raw'
docs = load_markdown_files(str(DATA_DIR))
print(f'Loaded {len(docs)} documents from {DATA_DIR}.')
if docs:
    preview_doc = docs[0]
    preview_text = preview_doc.get('text', '')
    meta = {k: v for k, v in preview_doc.items() if k != 'text'}
    print('Sample metadata:', meta)
    print('Preview snippet:', preview_text[:300], '...')

Loaded 5 documents from C:\Users\tomasz.makowski.2\Desktop\SemesterII\ComputationalIntelligence\Project\agentic-rag-architectures\data\raw.
Sample metadata: {'id': 'ff42879e-ca8b-4fbf-b961-030df96bbd35', 'filename': 'football_rules.md', 'path': 'C:\\Users\\tomasz.makowski.2\\Desktop\\SemesterII\\ComputationalIntelligence\\Project\\agentic-rag-architectures\\data\\raw\\football_rules.md'}
Preview snippet: # Comprehensive Overview of Football (Soccer) Rules: Structure, Gameplay, and Interpretations

## Introduction: The Logic and Governance of Football Laws
Football (known as soccer in some countries) is governed globally by the **Laws of the Game**, maintained by **IFAB (International Football Associ ...


## 2. Chunking


In [3]:
chunks = chunk_documents(docs, chunk_size=400, overlap=50)
print(f'Generated {len(chunks)} chunks.')
for chunk in chunks[:3]:
    snippet = chunk.get('text', '')[:150].replace('\n', ' ')
    print({'chunk_id': chunk.get('chunk_id'), 'metadata': chunk.get('metadata', {}), 'snippet': snippet})


[2025-11-25 23:33:01][INFO][src.chunking] Saved 22 chunks to ..\data\processed\chunks.json


Generated 22 chunks.
{'chunk_id': '295565fe-f348-4f70-ab98-9c39717e8020', 'metadata': {}, 'snippet': '# Comprehensive Overview of Football (Soccer) Rules: Structure, Gameplay, and Interpretations ## Introduction: The Logic and Governance of Football La'}
{'chunk_id': 'd2251b49-d357-498f-9a50-92bd14b79a03', 'metadata': {}, 'snippet': 'substitutes** (varies by competition) At least seven players are required to start a match. If a team drops below seven due to injuries or dismissals,'}
{'chunk_id': '885be910-f7c0-48dc-8cb0-67378dd770fc', 'metadata': {}, 'snippet': 'Handball (deliberate) A direct free kick may lead to a penalty if inside the defending penalty area. ### **Indirect Free Kick Offenses** Include: - Da'}


## 3. Embedding


In [4]:
embeddings, index_map = generate_embeddings(chunks, provider='openai')
print('Embeddings shape:', embeddings.shape)
print('Index map sample:', list(index_map.items())[:3])


[2025-11-25 23:33:01][INFO][src.embedder] EMBED | start | chunks=22
[2025-11-25 23:33:03][INFO][httpx] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-11-25 23:33:03][INFO][src.embedder] EMBED | processed batch 1/2 (16 chunks)
[2025-11-25 23:33:04][INFO][httpx] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-11-25 23:33:04][INFO][src.embedder] EMBED | processed batch 2/2 (6 chunks)
[2025-11-25 23:33:04][INFO][src.embedder] EMBED | provider=openai model=text-embedding-3-small
[2025-11-25 23:33:04][INFO][src.embedder] EMBED | chunks=22 batch_size=16 batches=2 embedding_dim=1536
[2025-11-25 23:33:04][INFO][src.embedder] EMBED | saved embeddings -> ..\embeddings\embeddings.npy
[2025-11-25 23:33:04][INFO][src.embedder] EMBED | saved index map -> ..\embeddings\embedding_index.json


Embeddings shape: (22, 1536)
Index map sample: [('295565fe-f348-4f70-ab98-9c39717e8020', 0), ('d2251b49-d357-498f-9a50-92bd14b79a03', 1), ('885be910-f7c0-48dc-8cb0-67378dd770fc', 2)]


## 4. Retrieval


In [5]:
query = 'How does acceleration in sprinting work?'
query_embedding = generate_query_embedding(query, provider='openai')
retrieved = retrieve(query_embedding, embeddings, index_map, chunks, k=5, threshold=0.5)
print('Retrieved chunks:')
for row in retrieved:
    preview = row['text'][:150].replace('\n', ' ')
    print({'chunk_id': row['chunk_id'], 'score': round(row['score'], 4), 'preview': preview})


[2025-11-25 23:33:04][INFO][src.embedder] EMBED | query embedding | provider=openai model=text-embedding-3-small
[2025-11-25 23:33:04][INFO][httpx] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-11-25 23:33:04][INFO][src.retriever] RETRIEVE | start | vectors=22 k=5 threshold=0.50
[2025-11-25 23:33:04][INFO][src.retriever] RETRIEVE | threshold filtering | threshold=0.50 passed=2
[2025-11-25 23:33:04][INFO][src.retriever] RETRIEVE | top_k selected | [{'chunk_id': '3a2ca109-39c5-43f3-8674-2e1ac3d2ee65', 'score': 0.6915}, {'chunk_id': '5b3abccd-8489-4948-aa4e-94ce68176066', 'score': 0.6143}]


Retrieved chunks:
{'chunk_id': '3a2ca109-39c5-43f3-8674-2e1ac3d2ee65', 'score': 0.6915, 'preview': '# Layered Dynamics of Sprinting Mechanics ## Acceleration Foundations Acceleration out of the blocks decides whether the entire race unfolds on schedu'}
{'chunk_id': '5b3abccd-8489-4948-aa4e-94ce68176066', 'score': 0.6143, 'preview': 'and indirectly keeps the pelvis neutral. Meanwhile, the shank angle should stay inside 50° through steps seven to nine. If one leg rotates to 55° whil'}


## 5. LLM Generation


In [6]:
start = time.perf_counter()
answer = generate_answer(query, retrieved, provider='openai')
elapsed_ms = (time.perf_counter() - start) * 1000
print('Answer:')
print(answer)
print(f'Time elapsed: {elapsed_ms:.2f} ms')
print(f'Token estimate: {len(answer.split())}')


[2025-11-25 23:33:04][INFO][src.llm_orchestrator] LLM | provider=openai model=gpt-5-nano context_chunks=2 context_chars=5103
[2025-11-25 23:33:04][INFO][src.llm_orchestrator] LLM | prompt_len=5291 approx_tokens=833
[2025-11-25 23:33:04][INFO][src.llm_orchestrator] LLM | Sending request to OpenAI question: How does acceleration in sprinting work?...
[2025-11-25 23:33:22][INFO][httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2025-11-25 23:33:22][INFO][src.llm_orchestrator] LLM | OpenAI request completed in 17950.69 ms
[2025-11-25 23:33:22][INFO][src.llm_orchestrator] LLM | OpenAI response: Acceleration in sprinting is the process of rapidly increasing forward velocity from the start through a coordinated force-to-ground pattern. Key points from the provided material:

- Start mechanics (out of the blocks): aim for a 42–45° torso angle; align the initial shin with the torso; ground contact around 0.19 seconds to deliver the forward impulse; block sp

Answer:
Acceleration in sprinting is the process of rapidly increasing forward velocity from the start through a coordinated force-to-ground pattern. Key points from the provided material:

- Start mechanics (out of the blocks): aim for a 42–45° torso angle; align the initial shin with the torso; ground contact around 0.19 seconds to deliver the forward impulse; block spacing about 45 cm front-to-front and 96 cm front-to-back; drive the rear knee past hip level to prevent braking and keep the pelvis under the shoulders.

- Early acceleration (steps 3–8): the center of mass should rise gradually rather than jumping to an upright position; keep the pelvis from drifting behind the shoulders to avoid braking forces; knee lift should stay within a 60° envelope relative to the hip.

- Force transmission and biomechanics: acceleration relies on a blend of concentric drive and elastic return; the athlete travels roughly 0.8 m per step for the first ten strides; cadence is about 4.5 strides per

## 6. End-to-End Test


In [7]:
def run_demo(question: str, top_k: int = 5) -> None:
    q_emb = generate_query_embedding(question, provider='openai')
    retrieved_chunks = retrieve(q_emb, embeddings, index_map, chunks, k=top_k, threshold=0.5)
    response = generate_answer(question, retrieved_chunks, provider='openai')
    print('Question:', question)
    print('Answer:', response)
    print('Retrieved chunk IDs:', [item['chunk_id'] for item in retrieved_chunks])
    print('Tokens:', len(response.split()))

run_demo('What strategies influence sprint acceleration phases?')


[2025-11-25 23:33:22][INFO][src.embedder] EMBED | query embedding | provider=openai model=text-embedding-3-small
[2025-11-25 23:33:23][INFO][httpx] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-11-25 23:33:23][INFO][src.retriever] RETRIEVE | start | vectors=22 k=5 threshold=0.50
[2025-11-25 23:33:23][INFO][src.retriever] RETRIEVE | threshold filtering | threshold=0.50 passed=3
[2025-11-25 23:33:23][INFO][src.retriever] RETRIEVE | top_k selected | [{'chunk_id': '3a2ca109-39c5-43f3-8674-2e1ac3d2ee65', 'score': 0.7304}, {'chunk_id': '5b3abccd-8489-4948-aa4e-94ce68176066', 'score': 0.6916}, {'chunk_id': '2ab3bd05-d1c1-41cb-8b09-f4f82906094a', 'score': 0.5249}]
[2025-11-25 23:33:23][INFO][src.llm_orchestrator] LLM | provider=openai model=gpt-5-nano context_chunks=3 context_chars=6872
[2025-11-25 23:33:23][INFO][src.llm_orchestrator] LLM | prompt_len=7073 approx_tokens=1109
[2025-11-25 23:33:23][INFO][src.llm_orchestrator] LLM | Sending request to OpenAI que

Question: What strategies influence sprint acceleration phases?
Answer: - Technical setup and blocks
  - Cue a 45° (or 42°) torso angle; ensure the shin line mirrors torso projection.
  - Block spacing: about 45 cm front-to-front and 96 cm front-to-back.
  - Drive the rear knee past hip level without the heel peeling outward.

- Early acceleration mechanics (steps 1–8)
  - Center of mass rises gradually from step 3; keep pelvis aligned (not behind the shoulders) to avoid braking.
  - Knee lift kept within a 60° envelope relative to the hip.
  - Shank angle stays inside about 50° through steps 7–9.
  - Maintain a stiff ankle/ankle position (benefits from dorsiflexion or a slight heel drop to aid elastic rebound).

- Force, cadence, and limb timing
  - Push through roughly 0.8 m per step in the first ten strides; cadence near 4.5 strides/s.
  - Arm mechanics: keep elbow angle under ~105°; use rhythm cues (thumb brushing the hip pocket) to maintain neutral pelvis and timing.
  - Avoid exc